In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [3]:
df=pd.read_csv("/content/Thyroid_Diff.csv")

In [4]:
df.shape

(383, 17)

In [5]:
df.head()

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Thyroid Function,Physical Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurred
0,27,F,No,No,No,Euthyroid,Single nodular goiter-left,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Indeterminate,No
1,34,F,No,Yes,No,Euthyroid,Multinodular goiter,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
2,30,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
3,62,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
4,62,F,No,No,No,Euthyroid,Multinodular goiter,No,Micropapillary,Multi-Focal,Low,T1a,N0,M0,I,Excellent,No


In [6]:
df.isnull().sum()

,0
Age,0
Gender,0
Smoking,0
Hx Smoking,0
Hx Radiothreapy,0
Thyroid Function,0
Physical Examination,0
Adenopathy,0
Pathology,0
Focality,0


In [7]:
df.nunique()

,0
Age,65
Gender,2
Smoking,2
Hx Smoking,2
Hx Radiothreapy,2
Thyroid Function,5
Physical Examination,5
Adenopathy,6
Pathology,4
Focality,2


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Age                   383 non-null    int64 
 1   Gender                383 non-null    object
 2   Smoking               383 non-null    object
 3   Hx Smoking            383 non-null    object
 4   Hx Radiothreapy       383 non-null    object
 5   Thyroid Function      383 non-null    object
 6   Physical Examination  383 non-null    object
 7   Adenopathy            383 non-null    object
 8   Pathology             383 non-null    object
 9   Focality              383 non-null    object
 10  Risk                  383 non-null    object
 11  T                     383 non-null    object
 12  N                     383 non-null    object
 13  M                     383 non-null    object
 14  Stage                 383 non-null    object
 15  Response              383 non-null    ob

In [9]:
df.head()

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Thyroid Function,Physical Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurred
0,27,F,No,No,No,Euthyroid,Single nodular goiter-left,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Indeterminate,No
1,34,F,No,Yes,No,Euthyroid,Multinodular goiter,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
2,30,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
3,62,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
4,62,F,No,No,No,Euthyroid,Multinodular goiter,No,Micropapillary,Multi-Focal,Low,T1a,N0,M0,I,Excellent,No


In [10]:
df['Recurred'].value_counts()

,count
Recurred,
No,275
Yes,108


In [11]:
#doing label encoding
object_class=df.select_dtypes(include='object').columns

encoders={}
le=LabelEncoder()
for i in object_class:
  df[i]=le.fit_transform(df[i])
  encoders[i]=le

In [12]:
# outliers
def outlier_remove(df, col):
    Q1 = df[col].quantile(0.25)  # Calculate Q1 for the specific column
    Q3 = df[col].quantile(0.75)  # Calculate Q3 for the specific column
    IQR = Q3 - Q1

    lower_bound = Q1 - (1.5 * IQR)
    upper_bound = Q3 + (1.5 * IQR)


    median=df[col].median()


    df[col]=df[col].apply(lambda x:median if x<lower_bound or x>upper_bound else x)

    return df

In [13]:
numerical_col=df.select_dtypes(exclude='object').columns

In [14]:
numerical_col

Index(['Age', 'Gender', 'Smoking', 'Hx Smoking', 'Hx Radiothreapy',
       'Thyroid Function', 'Physical Examination', 'Adenopathy', 'Pathology',
       'Focality', 'Risk', 'T', 'N', 'M', 'Stage', 'Response', 'Recurred'],
      dtype='object')

In [15]:
df['Recurred'].value_counts()

,count
Recurred,
0,275
1,108


In [16]:
x=df.drop('Recurred',axis=1)
y=df['Recurred']

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
smote=SMOTE(random_state=42)
x_new,y_new=smote.fit_resample(x_train,y_train)

In [19]:
models={
    "decison tree":DecisionTreeClassifier(),
    "random forest":RandomForestClassifier(),
    "logistic regression":LogisticRegression(),

}

In [20]:
cv_scores={}

#perform 5-fold cross validation for each model
for model_name,model in models.items():
  scores=cross_val_score(model,x_new,y_new,cv=8)
  cv_scores[model_name]=scores
  print(f"{model_name} cross validation accuracy:{np.mean(scores):.2f}")
  print("-"*50 )

decison tree cross validation accuracy:0.95
--------------------------------------------------
random forest cross validation accuracy:0.96
--------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

logistic regression cross validation accuracy:0.90
--------------------------------------------------


In [21]:
decison_tree=DecisionTreeClassifier(random_state=42)
random_forest=RandomForestClassifier(random_state=42)
logistic_regression=LogisticRegression(random_state=42)

In [22]:
#hyperparameter grids for RandomizedSearchCV

param_grid_dt={
    "criterion":["gini","entropy"],
    "max_depth":[None,5,10,15,20,30],
    "min_samples_split":[2,5,10],
}
param_grid_rf={
    "n_estimators":[100,200,300],
    "criterion":["gini","entropy"],
     "max_depth":[None,5,10,15,20,30],
    "min_samples_split":[2,5,10],
}
param_grid_lr={
    "C":[0.001,0.01,0.1,1,10,100],
    "penalty":["l1","l2"],
    "solver":["liblinear"],
}

In [23]:
#hyperparameter tuning for both models
#the below steps can be autyomated by using a for loop or by using a pipeine
#perform randomizedsearchcv for each model

random_search_dt=RandomizedSearchCV(estimator=decison_tree,param_distributions=param_grid_dt,n_iter=10,cv=5,scoring="accuracy",random_state=42)

random_search_rf=RandomizedSearchCV(estimator=random_forest,param_distributions=param_grid_rf,n_iter=10,cv=5,scoring="accuracy",random_state=42)

random_search_lr=RandomizedSearchCV(estimator=logistic_regression,param_distributions=param_grid_lr,n_iter=10,cv=5,scoring="accuracy",random_state=42)

In [24]:
random_search_dt.fit(x_new,y_new)
random_search_rf.fit(x_new,y_new)
random_search_lr.fit(x_new,y_new)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(random_state=42),
                   param_distributions={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                                        'penalty': ['l1', 'l2'],
                                        'solver': ['liblinear']},
                   random_state=42, scoring='accuracy')

In [25]:
print(random_search_dt.best_params_)
print(random_search_rf.best_params_)

{'min_samples_split': 10, 'max_depth': 30, 'criterion': 'entropy'}
{'n_estimators': 300, 'min_samples_split': 2, 'max_depth': 15, 'criterion': 'entropy'}


In [26]:
print(random_search_dt.best_score_)
print(random_search_rf.best_score_)

0.9608393477679765
0.9585672280139


In [27]:
#get the model with best score

best_model=None
best_score=0

if random_search_dt.best_score_>best_score:
  best_score=random_search_dt.best_score_
  best_model=random_search_dt.best_estimator_

if random_search_rf.best_score_>best_score:
  best_score=random_search_rf.best_score_
  best_model=random_search_rf.best_estimator_

In [28]:
print(best_model)
print(best_score)

DecisionTreeClassifier(criterion='entropy', max_depth=30, min_samples_split=10,
                       random_state=42)
0.9608393477679765


In [29]:
#save the best model
with open("Thyroid_model.sav","wb") as f:
  pickle.dump(best_model,f)

In [30]:
random_forest.fit(x_new,y_new)

RandomForestClassifier(random_state=42)

In [31]:
#evaluate on test data
y_test_pred=best_model.predict(x_test)
print("accuracy score\n",accuracy_score(y_test,y_test_pred))
print("confusion matrix\n",confusion_matrix(y_test,y_test_pred))
print("classification report\n",classification_report(y_test,y_test_pred))

accuracy score
 0.935064935064935
confusion matrix
 [[55  3]
 [ 2 17]]
classification report
               precision    recall  f1-score   support

           0       0.96      0.95      0.96        58
           1       0.85      0.89      0.87        19

    accuracy                           0.94        77
   macro avg       0.91      0.92      0.91        77
weighted avg       0.94      0.94      0.94        77



In [33]:
#building predictive model
input_data = (62,0,0,140,268,0,0,160,0,3.6,0,3,4,4,7,9)
# change the input data to a numpy array
input_data_as_numpy_array= np.asarray(input_data)

# reshape the numpy array as we are predicting for only on instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction =best_model.predict(input_data_reshaped)
print(prediction)

if (prediction[0]== 0):
  print('The Person do not has Thyroid')
else:
  print('The Person has Thyroid')

[1]
The Person has Thyroid


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
